# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [43]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD




In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [45]:
data = pd.read_csv('/content/gdrive/MyDrive/creditcard.csv')

In [46]:
data.shape

(284807, 31)

In [47]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [48]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Splitting Training and Labels

In [49]:
# Separating Input and Output Colmumns 
x = data.iloc[:, :-1].values  # Input Columns
y = data.iloc[:, -1:].values # Output Columns 


In [50]:
x.shape

(284807, 30)

In [51]:
y.shape

(284807, 1)

In [52]:
# Installed imbalanced-learn in order to use SMOTE  
# pip install -U imbalanced-learn

## As the data is very imbalanced, I have used SMOTE (Synthetic Minority Oversampling Technique) for oversampling

In [53]:
from imblearn.over_sampling import SMOTE

x, y = SMOTE().fit_resample(x, y)


print (x.shape, y.shape)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(568630, 30) (568630,)


In [54]:
def split_train_val_test(x, y, train, val, test):
    x_rows, x_col = x.shape
    y_rows = y.shape # Removed column temporarily 
    
    train_start = 0
    train_stop = int(x_rows*train)
    
    val_start = train_stop
    val_stop = int(x_rows*train)+int(x_rows*val)
    
    test_start = val_stop
    test_stop = None

    x_train = x[train_start:train_stop,] # Extracting X_Training Data 
    x_val = x[val_start:val_stop,]
    x_test = x[test_start:test_stop]
    
    y_train = y[train_start:train_stop,] # Extracting y_Training Data 
    y_val = y[val_start:val_stop,]
    y_test = y[test_start:test_stop]
    
    #Splitting Desicription 
    desc = f'''Splitting Discription:
    Training Data {int(train*100)}%
    Validation Data {int(val*100)}%
    Testing Data {int(val*100)}%
    
    Shape of Training Data {x.shape}
    Shape of Target Data {y.shape}
    
    Shape of x_train {x_train.shape}
    Shape of x_val {x_val.shape}
    Shape of x_test {x_test.shape}
     
    Shape of y_train {y_train.shape}
    Shape of y_val {y_val.shape}
    Shape of y_test {y_test.shape}
    
    '''
    
    return x_train, x_val, x_test, y_train, y_val, y_test, desc



x_train, x_val, x_test, y_train, y_val, y_test, desc = split_train_val_test(x,y,.5,.3,.2)





In [55]:
print (desc)

Splitting Discription:
    Training Data 50%
    Validation Data 30%
    Testing Data 30%
    
    Shape of Training Data (568630, 30)
    Shape of Target Data (568630,)
    
    Shape of x_train (284315, 30)
    Shape of x_val (170589, 30)
    Shape of x_test (113726, 30)
     
    Shape of y_train (284315,)
    Shape of y_val (170589,)
    Shape of y_test (113726,)
    
    
